In [1]:
import cv2 as cv
import pandas as pd
import tensorflow as tf
import numpy as np

import keras
from matplotlib import pyplot as plt
from textblob import TextBlob 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from datetime import date
from sklearn.neural_network import MLPClassifier
import requests

/Users/tudor/Documents/Work AI/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
polarity = [];
subjectivity = [];

polarity_mapping = {"Extremely Negative": -2,
                    "Negative": -1,
                    "Neutral": 0,
                    "Positive": 1,
                    "Extremely Positive": 2};

inv_polarity_mapping = {-2: "Extremely Negative",
                        -1: "Negative",
                        0: "Neutral",
                        1: "Positive",
                        2: "Extremely Positive"}

data_sentiments = []

sentiments = ["Extremely Negative",  "Negative", "Neutral", "Positive", "Extremely Positive"];

count = 0;
sentiment_search = "";


In [3]:
def find_link(urls):

    l = urls.split();
    print(l);

    links = [];
    contents = [];

    for i in l:
        try:
            links.append(requests.head(i).headers['location']);
            req = requests.get(i);
            print(req.content);
            contents.append(req.content);
        except ConnectionError:
            links.append("");
    return links;


def calc_sentiment(row):
    text = row["OriginalTweet"];
    blob = TextBlob(text)

    polarity.append(float(blob.sentiment.polarity));
    subjectivity.append(float(blob.sentiment.subjectivity));
    return row;

def map_sentiments(row):
    data_sentiments.append(polarity_mapping[row["Sentiment"]]);
    return row;

def filter_data(row):
    text = row["OriginalTweet"]
    text = text.strip();
    #print(text);
    text = text.split();
    #print(text);
    
    new_text = "";

    urls = "";

    for word in text:
        #print("word[0]: ", word[0]);
        if("https" in word):
            #print("word:", word);
            urls += word + " ";
        if(word[0] == '@' or word[0] == '#' or ("https" in word)): continue;

        new_text += (word + " ");

    row["OriginalTweet"] = new_text;
    #print(new_text);

    #print(find_link(urls));

    return row;

def print_lines(row):
    global count;

    if(count < 10 and sentiment_search == row["Sentiment"]):
        print("         ", end="")
        print(row["OriginalTweet"]);
        print()
        print();
        count += 1;
    return row;

In [4]:
df = pd.read_csv("Pandemic_NLP_train.csv");
#df = df.head();
display(df);

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know it's getting tough when @KameronWilds...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [5]:
#df = df.apply(filter_data, axis = 1);
df = df.apply(calc_sentiment, axis=1);
df = df.apply(map_sentiments, axis=1);

In [6]:
display(df);

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know it's getting tough when @KameronWilds...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [7]:
print(sentiments)
for sent in sentiments:
    sentiment_search = sent;
    print("Sentimet: ", sent);
    count = 0;
    df.apply(print_lines, axis=1);

['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
Sentimet:  Extremely Negative
         Me, ready to go at supermarket during the #COVID19 outbreak.



Not because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...



#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n


         with 100  nations inficted with  covid  19  the world must  not  play fair with china  100 goverments must demand  china  adopts new guilde  lines on food safty  the  chinese  goverment  is guilty of  being  irosponcible   with life  on a global scale


         This morning I tested positive for Covid 19. I feel ok, I have no symptoms so far but have been isolated since I found out about my possible exposure to the virus.  Stay home people and be pragmatic. I will keep you updated on how I'm doing ???? No panic. https://t.co/Lg7HVMZglZ


         @7SealsOf

In [8]:
print(polarity);
print(subjectivity)
print(data_sentiments);

split_data = [];

for i in range(len(polarity)):
    split_data.append((polarity[i], subjectivity[i], data_sentiments[i]));

[0.0, 0.16666666666666666, -0.2, 0.20000000000000004, -0.07777777777777777, 0.09833333333333334, 0.0, -0.03333333333333333, 0.02734375, 0.0, -0.03571428571428571, 0.02083333333333333, 0.17777777777777778, 0.0, -0.1, 0.27976190476190477, 0.0, 0.16, 0.2916666666666667, 0.0, 0.06727272727272726, 0.25, 0.36500000000000005, 0.43125, -0.15166666666666667, 0.11666666666666665, 0.15227272727272728, 0.20681818181818182, -0.19999999999999998, 0.016666666666666663, 0.35, 0.0, 0.0, 0.2708333333333333, -0.18125, -0.03333333333333333, 0.0, 0.25, -0.11952380952380952, 0.0, 0.5625, -0.19999999999999998, -0.125, 0.0, 0.0, 0.0, 0.275, -0.12833333333333333, 0.0, 0.28125, -0.08333333333333333, -0.05, 0.15, -0.04166666666666667, -0.08499999999999999, 0.65, -0.3666666666666667, 0.031565656565656554, 0.27999999999999997, 0.0, 0.0, 0.0, 0.0, 0.44999999999999996, 0.0, -0.75, 0.0, 0.09166666666666667, 0.25, -0.08806818181818182, 0.18, 0.35, 0.0, -0.033333333333333326, 0.21666666666666667, -0.07916666666666666, 

In [9]:
train, test = train_test_split(split_data, test_size=0.1);

In [10]:
print(train)

[(0.25833333333333336, 0.5583333333333333, 2), (0.6, 1.0, 2), (0.0, 0.0, -1), (-0.041666666666666664, 0.15064102564102563, -2), (-0.038095238095238106, 0.49523809523809526, 2), (0.16666666666666666, 0.6666666666666666, 2), (0.5, 0.5, -1), (0.3, 0.1, 1), (0.375, 0.75, 1), (0.0, 0.0, -2), (0.375, 0.375, -1), (0.0, 0.0, 1), (-0.19583333333333333, 0.3729166666666667, -1), (0.0, 0.0, 1), (0.0, 0.0, 0), (0.13636363636363635, 0.45454545454545453, -1), (0.15, 0.25, -2), (0.0, 0.0, -1), (-0.6, 0.8, -1), (0.0, 0.0, 1), (0.15625, 0.3333333333333333, 0), (0.0, 0.0, 0), (-0.3, 0.5, -1), (-0.25, 0.75, 2), (0.35, 0.30000000000000004, 1), (0.0, 0.0, 1), (-0.5, 1.0, -2), (-0.11666666666666667, 0.0, -1), (0.25, 1.0, -1), (0.10833333333333332, 0.40833333333333327, 1), (-0.13055555555555556, 0.44722222222222224, 1), (0.6, 0.75, 2), (0.45999999999999996, 0.76, 2), (0.13636363636363635, 0.45454545454545453, 0), (-0.1847222222222222, 0.3666666666666667, 2), (-0.3416666666666667, 0.5, -1), (0.0, 0.0, 0), (0.0

In [11]:
polarity = [];
subjectivity = [];
data_sentiments = [];

for x, y, z in train:
    polarity.append(x);
    subjectivity.append(y);
    data_sentiments.append(z);

print(polarity)

[0.25833333333333336, 0.6, 0.0, -0.041666666666666664, -0.038095238095238106, 0.16666666666666666, 0.5, 0.3, 0.375, 0.0, 0.375, 0.0, -0.19583333333333333, 0.0, 0.0, 0.13636363636363635, 0.15, 0.0, -0.6, 0.0, 0.15625, 0.0, -0.3, -0.25, 0.35, 0.0, -0.5, -0.11666666666666667, 0.25, 0.10833333333333332, -0.13055555555555556, 0.6, 0.45999999999999996, 0.13636363636363635, -0.1847222222222222, -0.3416666666666667, 0.0, 0.0, 0.15, 0.0, 0.0, 0.0, 0.15000000000000002, 0.20000000000000004, -0.2666666666666667, -0.1, 0.5, 0.03333333333333333, 0.5, 0.0, 0.0, -0.024999999999999994, 0.5, 0.0, 0.12916666666666665, 0.016666666666666666, 0.18333333333333332, 0.16666666666666669, 0.35000000000000003, 0.0, 0.3619047619047619, 0.0, -0.19999999999999998, 0.13636363636363635, 0.10909090909090909, 0.0, -0.5, 0.0, 0.5, 0.3375, -0.16666666666666666, 0.24479166666666669, -0.35416666666666663, -0.175, 0.00888888888888888, -0.075, 0.0, 0.0, -0.35625, 0.0, 0.0, 0.6666666666666666, 0.08750000000000001, 0.2249999999

In [12]:
# device = torch.device("mps");
# print(device);

In [13]:
train_f = [];
train_l = data_sentiments;

for i in range(len(polarity)):
    train_f.append(polarity[i]);
    train_f.append(subjectivity[i]);

train_f = np.array(train_f).reshape(-1, 2);
train_l = np.array(train_l);

In [14]:
# y_tensor_list = [];

# for x in train_l:
#     if(x == -2):
#         y_tensor_list.append(1.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#     if(x == -1):
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(1.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#     if(x == 0):
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(1.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#     if(x == 1):
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(1.0);
#         y_tensor_list.append(0.0);
#     if(x == 2):
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(0.0);
#         y_tensor_list.append(1.0);



In [15]:
# y_tensor_list = np.array(y_tensor_list);
# y_tensor_list = y_tensor_list.reshape(-1, 5);

In [16]:
# X_tensor = torch.tensor(train_f, dtype=torch.float32).to(device);
# y_tensor = torch.tensor(y_tensor_list, dtype=torch.float32).to(device);

In [17]:
# print(len(X_tensor));
# print(len(y_tensor));

In [18]:
# model = nn.Sequential(
#     nn.Linear(2, 6),
#     nn.Softmax(),
#     nn.Linear(6, 6),
#     nn.Softmax(),
#     nn.Linear(6, 5),
#     #nn.Si(),
# )

# model.to(device);

In [19]:
# epochs = 100;
# batch_size = 10;

In [20]:
# loss_fn   = nn.L1Loss()  # binary cross entropy
# optimizer = optim.Adam(model.parameters(), lr=0.000001)

In [21]:
# for epoch in range(epochs):
#     for i in range(0, len(X_tensor), batch_size):
#         if(i % 10000 == 0): 
#             print(i / len(X_tensor) * 100, '% epoch', epoch);
            
#         Xbatch = X_tensor[i:i+batch_size].to(device)
#         y_pred = model(Xbatch).to(device);
#         ybatch = y_tensor[i:i+batch_size].to(device);
        
#         #print(y_pred);
#         #print(ybatch);

#         loss = loss_fn(y_pred, ybatch).to(device);
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     print(f'Finished epoch {epoch}, latest loss {loss}')
#     model_scripted = torch.jit.script(model)
#     model_scripted.save('./model_saves/model_' + str(epoch) + "_" + str(date.today()) + ".pt"); 
#     print("Avarege:", get_accuracy(model, test, device));

In [22]:
test_f = [];
test_l = [];

for x, y, z in test:
    test_f.append(x);
    test_f.append(y);
    test_l.append(z);

test_f = np.array(test_f).reshape(-1, 2);

In [23]:
modell = MLPClassifier(verbose=True, learning_rate='adaptive', tol=0.0000001, max_iter=1000).fit(train_f, train_l);

Iteration 1, loss = 1.52514206
Iteration 2, loss = 1.47680835
Iteration 3, loss = 1.47371101
Iteration 4, loss = 1.47249541
Iteration 5, loss = 1.47131661
Iteration 6, loss = 1.47053372
Iteration 7, loss = 1.47002089
Iteration 8, loss = 1.46948938
Iteration 9, loss = 1.46896680
Iteration 10, loss = 1.46864911
Iteration 11, loss = 1.46831368
Iteration 12, loss = 1.46784480
Iteration 13, loss = 1.46785402
Iteration 14, loss = 1.46750525
Iteration 15, loss = 1.46720280
Iteration 16, loss = 1.46701983
Iteration 17, loss = 1.46706032
Iteration 18, loss = 1.46683709
Iteration 19, loss = 1.46666771
Iteration 20, loss = 1.46631399
Iteration 21, loss = 1.46635333
Iteration 22, loss = 1.46630795
Iteration 23, loss = 1.46597070
Iteration 24, loss = 1.46580596
Iteration 25, loss = 1.46553631
Iteration 26, loss = 1.46554325
Iteration 27, loss = 1.46522431
Iteration 28, loss = 1.46510801
Iteration 29, loss = 1.46530253
Iteration 30, loss = 1.46492649
Iteration 31, loss = 1.46487213
Iteration 32, los

In [24]:
test_l = np.array(test_l);

print(modell.predict(test_f));
print(test_l);

print(f1_score(modell.predict(test_f), test_l, average="micro"));

[-1 -1 -1 ...  2  0  1]
[-1 -2 -1 ...  1  1  1]
0.3627308066083576


In [25]:
real_data = pd.read_csv("Pandemic_NLP_test_.csv", encoding="UTF-8");
ans = pd.DataFrame();
ans["OriginalTweet"] = real_data["OriginalTweet"];

In [26]:
polarity = [];
subjectivity = [];

In [27]:
real_data = real_data.apply(filter_data, axis=1);
real_data = real_data.apply(calc_sentiment, axis=1);

In [28]:
print(polarity);
print(subjectivity);

features = []

for i in range(len(polarity)):
    features.append(polarity[i]);
    features.append(subjectivity[i]);

features = np.array(features);

features = features.reshape(-1, 2);

[1.0, 0.33035714285714285, 0.0, 0.0, -0.075, 0.25, 0.0, 0.0, 0.15666666666666668, 4.625929269271485e-18, 0.33333333333333337, 0.0, 0.0, 0.2, 0.0, -0.8749999999999998, -0.10370370370370373, 0.27, -0.002777777777777777, 0.0, 0.2795833333333333, -0.3216269841269841, -0.008333333333333331, 0.45555555555555555, 0.0755952380952381, 0.0, 0.04999999999999999, -0.15714285714285714, 0.0, 0.0, -0.35714285714285715, -0.03333333333333333, 0.04545454545454545, -0.020833333333333336, 0.0, 0.0, -0.5333333333333333, 0.17600000000000002, 0.55, 0.7125, 0.08571428571428572, 0.0, 0.07619047619047618, 0.0, 0.06999999999999999, 0.1, 0.0, -0.15714285714285714, -0.23214285714285715, -0.23333333333333334, 0.0, 0.25, -0.05, 0.0, -0.03333333333333333, -0.03333333333333332, -0.24444444444444444, 0.0, 0.0, 0.5, -0.13214285714285715, -0.1875, 0.0, 0.0, 0.34545454545454546, -0.25, 0.18888888888888888, -0.140625, 0.0, 0.21875, 0.0, -0.23214285714285715, -0.12000000000000002, 0.0, 0.23125, 0.16333333333333333, 0.15, 0.

In [29]:
print(modell.predict(features));

[2 1 0 ... 1 1 2]


In [30]:
pred = modell.predict(features);
string_pred = [];

for i in range(len(pred)):
    string_pred.append(inv_polarity_mapping[pred[i]]);

In [31]:
ans["Sentiment"] = string_pred;
ans.to_csv("ans.csv", index=False);
display(ans)

,OriginalTweet,Sentiment
0,COVID-19 might be presenting online shopping p...,Extremely Positive
1,My right wing coo coo father in law was tellin...,Positive
2,I cannot decide if I am the smartest person in...,Neutral
3,Why are people stock piling what s wrong wit...,Negative
4,Show me where the eggs are. That's all I need ...,Negative
...,...,...
3793,You can tell this episode of Here's Lucy is fr...,Positive
3794,"You can watch my 11-minute video on ""Tiny Habi...",Negative
3795,"You know, normally I don't tweet but I am low-...",Positive
3796,You would have thought that with everyone so w...,Positive


In [32]:
string = "happy https://happy.com";
string1 = "happy";



blob = TextBlob(string);
blob1 = TextBlob(string1);

print(blob.sentiment)

print(blob1.sentiment)

Sentiment(polarity=0.8, subjectivity=1.0)
Sentiment(polarity=0.8, subjectivity=1.0)


In [33]:
string = "beutiful happy sucks";
string1 = "happy beutiful sucks";
string2 = "sucks beutiful happy";
string3 = "beutiful sucks happy";


blob = TextBlob(string);
blob1 = TextBlob(string1);
blob2 = TextBlob(string2);
blob3 = TextBlob(string3);


print(blob.sentiment)
print(blob1.sentiment)
print(blob2.sentiment)
print(blob3.sentiment)



Sentiment(polarity=0.25, subjectivity=0.65)
Sentiment(polarity=0.25, subjectivity=0.65)
Sentiment(polarity=0.25, subjectivity=0.65)
Sentiment(polarity=0.25, subjectivity=0.65)
